[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NaiaraSPinto/VegMapper/blob/master/gedi/gedi_l2a_arset.ipynb)


# 1- Install packages and import dependencies

In [11]:
%pip install -q geoviews earthaccess harmony-py

Note: you may need to restart the kernel to use updated packages.


In [2]:
from harmony import BBox, Client, Collection, Request, CapabilitiesRequest
from datetime import datetime
import json
import earthaccess
import geopandas as gp
import os
from IPython.display import JSON
import h5py
import pandas as pd
from shapely.geometry import Point
import ast
import subprocess
import requests
import tempfile

# 2- Authenticate with your NASA Earth Data credentials

In [3]:
auth = earthaccess.login(persist=True)

# 3- Define your Area Of Interest by dragging a GEOJSON into your local folder
You may need to simplify your polygon

In [4]:
my_site = input("Enter the path to your GeoJSON polygon file: ")

try:
    with open(my_site) as f:
        geojson_polygon = json.load(f)
    print("GeoJSON loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {my_site}")
except json.JSONDecodeError:
    print(f"Invalid GeoJSON format in file: {my_site}")

GeoJSON loaded successfully.


# 4- Define the temporal range of your search
GEDI data availability:
*   April 2019 - March 2023
*   April 2024 - Present



In [5]:
start_str = input("Enter start date (YYYY-MM-DD): ")
stop_str = input("Enter stop date (YYYY-MM-DD): ")

# Convert input strings to datetime objects
try:
    temporal_range = {
        'start': datetime.strptime(start_str, "%Y-%m-%d"),
        'stop': datetime.strptime(stop_str, "%Y-%m-%d")
    }
    print("Temporal range set to:", temporal_range)
except ValueError:
    print("Invalid date format. Please use YYYY-MM-DD.")
output_folder = input("Enter the output folder where you wish to save your results:")

Temporal range set to: {'start': datetime.datetime(2024, 12, 1, 0, 0), 'stop': datetime.datetime(2024, 12, 31, 0, 0)}


# 5- Call the Harmony Service to subset GEDI granules for our Area Of Interest

In [6]:
harmony_client = Client(auth=(auth.username, auth.password))
capabilities_request = CapabilitiesRequest(short_name='GEDI02_A')
capabilities = harmony_client.submit(capabilities_request)
concept_id = capabilities['conceptId']

request = Request(
    collection = Collection(id=concept_id),
    shape = my_site,
    temporal = temporal_range
)
task = harmony_client.submit(request)
print(f'Harmony request ID: {task}')
print(f'Processing your Harmony request:')
task_json = harmony_client.result_json(task, show_progress=True)

 [ Processing:   0% ] |                                                   | [/]


Harmony request ID: 26215715-f9d2-4291-a6a6-4ed604da3482
Processing your Harmony request:


 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   9% ] |####                                               | [-]
 [ Processing:   9% ] |####                                               | [\]
 [ Processing:   9% ] |####                                               | [|]
 [ Processing:   9% ] |####                                               | [/]
 [ Processing:   9% ] |####             

# 6- Save the list of subsetted files
The list contains the remote location of files containing GEDI shots in H5 format. Here, we are only retrieving a list of file locations, not the files themselves.

In [7]:
h5_files = [link['href'] for link in task_json['links'] if link['href'].endswith('.h5')]
with open(os.path.join(output_folder, 'h5_files.txt'), 'w') as f:
    for h5_file in h5_files:
        f.write(h5_file + '\n')

print("H5 file links saved to h5_files.txt")

H5 file links saved to h5_files.txt


# 7- User-defined quality and height criteria
Here are some commonly-used criteria:
*   Min RH95 can be used to **exclude bare ground**
*   Sensitivity is the the maximum canopy cover through which the LiDAR can detect the ground with 90% probability, and it is useful in areas of **dense vegetation**
*   Night time shots can be selected to maximize **signal-to-noise ratio**












In [8]:
#sensitivity
while True:
    min_sens_input = input("Enter minimum sensitivity value (0.0-0.9): ")
    try:
        min_sens = float(min_sens_input)
        # Check if the float is within the desired range
        if 0.0 <= min_sens <= 0.9:
            print(f"Minimum sensitivity set to: {min_sens}")
            break  # Exit the loop if input is valid
        else:
            print("Error: The value must be between 0.0 and 0.9.")
    except ValueError:
        print("Error: Invalid input. Please enter a numerical value.")

#min RH95
while True:
    min_height_input = input("Enter minimum RH95 value (0-5): ")
    try:
        min_height = int(min_height_input)
        # Check if the integer is within the desired range
        if 0 <= min_height <= 5:
            print(f"Minimum height set to: {min_height}")
            break # Exit the loop if input is valid
        else:
            print("Error: The value must be between 0 and 5.")
    except ValueError:
        print("Error: Invalid input. Please enter a numerical value.")

#night time only
try:
    night_only = ast.literal_eval(input("Enter True or False for night only filtering: "))
    if isinstance(night_only, bool):
        print(f"The boolean value is: {night_only}")
        print(f"The type is: {type(night_only)}")
    else:
        print("Invalid input. The input must be 'True' or 'False'.")
except (ValueError, SyntaxError):
    print("Invalid input. The input must be 'True' or 'False'.")




Minimum sensitivity set to: 0.8
Minimum height set to: 4
The boolean value is: False
The type is: <class 'bool'>


# 8- Function to filter GEDI L2A shots
For each H5 file, download from the DAAC, apply the user-specified filtering, and save results as a local CSV.

In [9]:
#This is the main function
def extract_gedi_rh_metrics_from_urls(file_list_txt, csv_file, beams=None,
                                      min_sensitivity=0.9, min_rh95=0,
                                      night=False):
    """
    Reads a list of HTTPS GEDI HDF5 file URLs, downloads each file completely,
    extracts RH metrics for specified beams, filters by sensitivity, RH95,
    and optionally by solar elevation (nighttime), and saves all shots to a CSV.

    Parameters
    ----------
    file_list_txt : str
        Path to a text file containing one HTTPS URL per line.
    csv_file : str
        Output CSV file path.
    beams : list, optional
        List of beam names to extract. Defaults to the four full-power beams.
    min_sensitivity : float, optional
        Minimum acceptable sensitivity (default: 0.95)
    min_rh95 : float, optional
        Minimum acceptable RH95 value (default: 0)
    night : bool, optional
        If True, select only shots where solar_elevation < 0 (nighttime)
    """
    if beams is None:
        beams = ['BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']

    # Read list of URLs
    with open(file_list_txt, 'r') as f:
        h5_urls = [line.strip() for line in f if line.strip()]

    records = []

    for url in h5_urls:
        print(f"\n📥 Downloading {url} ...")
        try:
            response = requests.get(url, timeout=300)
            response.raise_for_status()

            with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
                tmp.write(response.content)
                tmp_path = tmp.name

            with h5py.File(tmp_path, 'r') as f:
                for beam_name in beams:
                    if beam_name not in f:
                        print(f"  ⚠️ Beam {beam_name} not found in {url}")
                        continue

                    beam = f[beam_name]

                    # Check datasets required for GEDI L2A v2.1+
                    required = ['lat_lowestmode', 'lon_lowestmode', 'rh', 'shot_number', 'solar_elevation']
                    if not all(k in beam for k in required):
                        print(f"  ⚠️ Missing required datasets in {beam_name}")
                        continue

                    # Try to access sensitivity safely
                    sensitivity = None
                    if 'QA' in beam and 'sensitivity' in beam['QA']:
                        sensitivity = beam['QA/sensitivity'][:]
                    elif 'sensitivity' in beam:
                        sensitivity = beam['sensitivity'][:]
                    else:
                        print(f"  ⚠️ Sensitivity missing in {beam_name}, skipping beam")
                        continue

                    lat = beam['lat_lowestmode'][:]
                    lon = beam['lon_lowestmode'][:]
                    shot_num = beam['shot_number'][:]
                    rh = beam['rh'][:]
                    solar = beam['solar_elevation'][:]

                    # Extract selected RH metrics
                    rh25 = rh[:, 25]
                    rh50 = rh[:, 50]
                    rh75 = rh[:, 75]
                    rh95 = rh[:, 95]

                    # Apply filters
                    mask = (sensitivity >= min_sensitivity) & (rh95 >= min_rh95)
                    if night:
                        mask &= (solar < 0)

                    if not mask.any():
                        print(f"  ⚙️ No shots passed filters in {beam_name}")
                        continue

                    df = pd.DataFrame({
                        "source_url": url,
                        "beam": beam_name,
                        "shot_number": shot_num[mask],
                        "latitude": lat[mask],
                        "longitude": lon[mask],
                        "rh25": rh25[mask],
                        "rh50": rh50[mask],
                        "rh75": rh75[mask],
                        "rh95": rh95[mask],
                        "sensitivity": sensitivity[mask],
                        "solar_elevation": solar[mask]
                    })

                    records.append(df)

        except requests.exceptions.RequestException as e:
            print(f"❌ Download failed for {url}: {e}")
        except Exception as e:
            print(f"❌ Error processing {url}: {e}")
        finally:
            if 'tmp_path' in locals() and os.path.exists(tmp_path):
                os.remove(tmp_path)

    # Combine all dataframes
    if records:
        shots_df = pd.concat(records, ignore_index=True)
        shots_df.to_csv(csv_file, index=False)
        print(f"\n✅ Total shots saved: {len(shots_df):,}")
        print(f"✅ Output file: {csv_file}")
    else:
        print("\n⚠️ No valid shots extracted.")


# 9- Run the filter function
Results will be saved in a CSV file that can be downloaded into your computer

In [11]:
# Path to your GEDI L2A file subset by Harmony
gedi_file = input("Enter the path to the H5 file list")
out_csv_file = input("Enter the path to the output CSV file")

#List of Full Power Beams
full_power_beams = ['BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']

#Get the filtering variables
min_sens_input = float(min_sens_input)
min_height_input = int(min_height_input)
night_only = bool(night_only)

#Run the function to filter H5 file and extract RH metrics
extract_gedi_rh_metrics_from_urls(gedi_file, out_csv_file, beams=full_power_beams,
                                      min_sensitivity=min_sens_input, min_rh95=min_height_input,
                                      night=night_only)


📥 Downloading https://harmony.earthdata.nasa.gov/service-results/harmony-prod-staging/public/26215715-f9d2-4291-a6a6-4ed604da3482/120972297/GEDI02_A_2024337124915_O33835_01_T01605_02_004_02_V002_subsetted.h5 ...

📥 Downloading https://harmony.earthdata.nasa.gov/service-results/harmony-prod-staging/public/26215715-f9d2-4291-a6a6-4ed604da3482/120972298/GEDI02_A_2024338120144_O33850_01_T08123_02_004_02_V002_subsetted.h5 ...

📥 Downloading https://harmony.earthdata.nasa.gov/service-results/harmony-prod-staging/public/26215715-f9d2-4291-a6a6-4ed604da3482/120972300/GEDI02_A_2024342102412_O33911_01_T00197_02_004_02_V002_subsetted.h5 ...
  ⚙️ No shots passed filters in BEAM0101
  ⚙️ No shots passed filters in BEAM0110
  ⚙️ No shots passed filters in BEAM1000
  ⚙️ No shots passed filters in BEAM1011

📥 Downloading https://harmony.earthdata.nasa.gov/service-results/harmony-prod-staging/public/26215715-f9d2-4291-a6a6-4ed604da3482/120972301/GEDI02_A_2024345093353_O33957_01_T05675_02_004_02_V002_s